# Credit Card Fraud Detection

The previous exercises made you take a closer look at all the different parts of a neural network: 
* the architecture of a sequential Dense Neural Network, 
* the compilation method
* the fitting.

Let's now work on a real-life dataset that has **a lot of data**!

**The dataset: `Credit Card Transactions`**

For this open challenge, you will `work with data extracted from credit card transactions`. 

As this is `sensitive data`, only 3 columns are known out of a total 31: the rest have been transformed to `anonymize` them (in fact, they are `PCA projections of initial data`).

The 3 known columns are:

* `TIME`: the time elapsed between the transaction and the first transaction in the dataset
* `AMOUNT`: the amount of the transaction
* `CLASS` (our target): 
    * `0 : valid transaction` 
    * `1 : fraudulent transaction`

❓ **Question** ❓ Start by downloading the dataset:
* on the Kaggle website [here](https://www.kaggle.com/mlg-ulb/creditcardfraud) 
* or from our [URL](https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/creditcard.csv) 

Load data to create `X` and `y`

In [3]:
# YOUR CODE HERE

import pandas as pd

# Loada data from csv file
data = pd.read_csv('creditcard.csv')

# Print the first 5 rows of the dataframe.
data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Create X and y from the data
X = data.drop(['Class'], axis=1)
y = data['Class']

# Print the shapes of X & y
print(X.shape)
print(y.shape)


(284807, 30)
(284807,)


## 1. Rebalancing classes

In [5]:
# Let's check class balance
pd.Series(y).value_counts(normalize=True)


0    0.998273
1    0.001727
Name: Class, dtype: float64

☝️ in this `fraud detection` challenge, **the classes are extremely imbalanced**:
* 99.8 % of normal transactions
* 0.2 % of fraudulent transactions

**We won't be able to detect cases of fraud unless we apply some serious rebalancing strategies!**

❓ **Question** ❓
1. **First**, create three separate splits `Train/Val/Test` from your dataset. It is extremely important to keep validation and testing sets **unbalanced** so that when you evaluate your model, it is done in true conditions, without data leakage. Keep your test set for the very last cell of this notebook!

&nbsp;
2. **Second**, rebalance your training set (and only this one). You have many choices:

- Simply oversample the minority class randomly using plain Numpy functions (not the best option since you are duplicating rows and hence creating data leakage)
- Or use <a href="https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/">**`Synthetic Minority Oversampling Technique - SMOTE`**</a> to generate new datapoints by weighting the existing ones
- In addition, you can also try a <a href="https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/">**`RandomUnderSampler`**</a> to downsample the majority class a little bit

In [6]:
# YOUR CODE HERE
# Create training, validation and test sets
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, random_state=42)

# Print the shapes of X & y for each set
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)


/Users/tanushrinayak/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(160203, 30)
(53402, 30)
(71202, 30)
(160203,)
(53402,)
(71202,)


In [8]:
# Rebalance the training data using SMOTE
from imblearn.over_sampling import SMOTE

# Create the SMOTE object
smote = SMOTE(random_state=42)

# Train data will be used to fit the SMOTE object
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


## 2. Neural Network iterations

Now that you have rebalanced your classes, try to fit a neural network to optimize your test score. Feel free to use the following hints:

- Normalize your inputs!
    - Use preferably a [`Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) layer inside the model to "pipeline" your preprocessing within your model. 
    - Or use sklearn's [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) outside of your model, applied your `X_train` and `X_val` and `X_test`.
- Make your model overfit, then regularize  it using:
    - Early Stopping criteria 
    - [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers
    - or [`regularizers`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers) layers
- 🚨 Think carefully about the metrics you want to track and the loss function you want to use!


In [9]:
# YOUR CODE HERE
# Fit a neural network model after Normalizing the data
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Standardize X_train_smote, X_val, X_test
scaler = StandardScaler()
X_train_smote = scaler.fit_transform(X_train_smote)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Create the neural network
model = Sequential()
model.add(Dense(32, activation="relu", input_shape=(30,)))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Fit the model using Early Stopping
model.fit(
    X_train_smote,
    y_train_smote,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[EarlyStopping(patience=5)],
)

# Evaluate the model on the test data
model.evaluate(X_test, y_test)


2023-11-14 14:16:37.642837: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
9996/9996 [==============================] - 14s 1ms/step - loss: 0.0260 - accuracy: 0.9913 - val_loss: 0.0129 - val_accuracy: 0.9979
Epoch 2/100
9996/9996 [==============================] - 14s 1ms/step - loss: 0.0063 - accuracy: 0.9983 - val_loss: 0.0117 - val_accuracy: 0.9988
Epoch 3/100
9996/9996 [==============================] - 14s 1ms/step - loss: 0.0044 - accuracy: 0.9989 - val_loss: 0.0146 - val_accuracy: 0.9980
Epoch 4/100
9996/9996 [==============================] - 14s 1ms/step - loss: 0.0037 - accuracy: 0.9990 - val_loss: 0.0137 - val_accuracy: 0.9984
Epoch 5/100
9996/9996 [==============================] - 14s 1ms/step - loss: 0.0032 - accuracy: 0.9992 - val_loss: 0.0169 - val_accuracy: 0.9984
Epoch 6/100
9996/9996 [==============================] - 14s 1ms/step - loss: 0.0027 - accuracy: 0.9993 - val_loss: 0.0210 - val_accuracy: 0.9969
Epoch 7/100
2226/2226 [==============================] - 2s 906us/step - loss: 0.0160 - accuracy: 0.9978


[0.01603597030043602, 0.997780978679657]

### 🧪 Test your score

Store below your real test performance on a (`X_test`, `y_test`) representative sample of the original unbalanced dataset into `precision` and `recall` variables.

In [10]:
# YOUR CODE HERE

# Evaluate the model on the test data and
# print the 'precision', 'recall' and 'f1-score'
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score,
)

# Predict on the test data
y_pred = model.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred.round()))

# Print the precision score
precision = precision_score(y_test, y_pred.round())
print("Precision:", precision)

# Print the recall score
recall = recall_score(y_test, y_pred.round())
print("Recall:", recall)

# Print the f1 score
f1 = f1_score(y_test, y_pred.round())
print("F1:", f1)


2226/2226 [==============================] - 2s 977us/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71079
           1       0.42      0.78      0.55       123

    accuracy                           1.00     71202
   macro avg       0.71      0.89      0.77     71202
weighted avg       1.00      1.00      1.00     71202

Precision: 0.42290748898678415
Recall: 0.7804878048780488
F1: 0.5485714285714286


In [11]:
from nbresult import ChallengeResult

result = ChallengeResult('solution',
    precision=precision,
    recall=recall,
    fraud_number=len(y_test[y_test == 1]),
    non_fraud_number=len(y_test[y_test == 0]),
)

result.write()
print(result.check())



============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/tanushrinayak/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/tanushrinayak/code/tanushrin/06-Deep-Learning/02-Optimizer-loss-and-fitting/data-credit-card-challenge/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_solution.py::TestSolution::test_is_score_good_enough PASSED         [ 50%]
test_solution.py::TestSolution::test_is_test_set_representative PASSED   [100%]

============================== 2 passed in 0.13s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master



## 🏁 Optional: Read Google's solution for this challenge
Congratulations for finishing all challenges for this session!

To conclude, take some time to read Google's own solution direcly [on Colab here](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb). 

You will discover interesting techniques and best practices